In [1]:
import os
import pandas as pd
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [2]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-15.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [3]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [4]:
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialise lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that we have all the data, put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame 
popularity_df.head()


,artist_name,track_name,popularity_score
0,Taylor Swift,Cruel Summer,99
1,Noah Kahan,Stick Season,98
2,Benson Boone,Beautiful Things,97
3,Jack Harlow,Lovin On Me,99
4,Beyoncé,TEXAS HOLD 'EM,0


In [5]:
df['popularity_score'] = popularity_df['popularity_score']

In [6]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [7]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,15-02-2024,1,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,3,376,318169,99
1,15-02-2024,2,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,133,314704,98
2,15-02-2024,3,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,2,28,304743,97
3,15-02-2024,4,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,4,98,259052,99
4,15-02-2024,5,Beyoncé,TEXAS HOLD 'EM,7AYmToZ5y67fBjq4RLFbuq,Parkwood Entertainment/Columbia,5,6,4,238907,0


Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [8]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [9]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

Need to change "=" to "0", remove "+", and handle "RE"

In [10]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "NEW" at position 6
Non-numeric values found:
      rank rank_change                                       artist_title  \
6       7         NEW       ¥$ - CARNIVAL (w/ Kanye West, Ty Dolla $ign)   
12     13         NEW           ¥$ - BURN (w/ Kanye West, Ty Dolla $ign)   
31     32         NEW     ¥$ - BACK TO ME (w/ Kanye West, Ty Dolla $ign)   
38     39         NEW       ¥$ - FUK SUMN (w/ Kanye West, Ty Dolla $ign)   
61     62         NEW          ¥$ - STARS (w/ Kanye West, Ty Dolla $ign)   
71     72         NEW          ¥$ - DO IT (w/ Kanye West, Ty Dolla $ign)   
84     85         NEW           ¥$ - PAID (w/ Kanye West, Ty Dolla $ign)   
95     96         NEW        ¥$ - TALKING (w/ Kanye West, Ty Dolla $ign)   
129   130          RE                Taylor Swift - ‘tis the damn season   
130   131          RE               Taylor Swift - 22 (Taylor's Version)   
131   132          RE                            

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_29134/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [11]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [12]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Taylor Swift - Cruel Summer,2,2.0,35283.0,1883256,82067,54009702
1,Noah Kahan - Stick Season,-1,19.0,-8271.0,2240168,-24428,24270527
2,Benson Boone - Beautiful Things,-1,7.0,11822.0,2011597,36529,5870484


In [13]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [14]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [15]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [16]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,15-02-2024,Taylor Swift,Cruel Summer,1,3,2,1,2.0,376,99,318169,35283.0,1883256,82067,54009702,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
1,15-02-2024,Noah Kahan,Stick Season,2,1,-1,1,19.0,133,98,314704,-8271.0,2240168,-24428,24270527,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
2,15-02-2024,Benson Boone,Beautiful Things,3,2,-1,2,7.0,28,97,304743,11822.0,2011597,36529,5870484,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,15-02-2024,Jack Harlow,Lovin On Me,4,4,0,1,72.0,98,99,259052,1817.0,1857133,-25622,30920955,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
4,15-02-2024,Beyoncé,TEXAS HOLD 'EM,5,6,1,5,NaN,4,0,238907,35446.0,744685,238907,744685,Parkwood Entertainment/Columbia,7AYmToZ5y67fBjq4RLFbuq


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          42 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  174 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [18]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.tail()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
1995,14-02-2024,Snow Patrol,Chasing Cars,196,-1,RE,130,NaN,446,85,50875,NaN,50875,50875,13095697,Polydor Records,5hnyJvgoWiQUYZttV4wXy6
1996,14-02-2024,Noah Kahan,Forever,197,156,-41,55,NaN,6,81,50703,-3496.0,377491,50703,377491,Mercury Records/Republic Records,0FlwhvrncUKrEAhzunmCKm
1997,14-02-2024,4batz,act ii: date @ 8,198,195,-3,132,NaN,19,89,50539,1086.0,352757,-5200,941973,4Batz,4HMop4Re0iucehmF7mgV27
1998,14-02-2024,Taylor Swift,tolerate it,199,-1,RE,26,NaN,12,80,50454,NaN,50454,50454,860180,Taylor Swift,0PurA4JVJ8YQgSVopY8fn6
1999,14-02-2024,Taylor Swift,We Are Never Ever Getting Back Together (Taylo...,200,-1,RE,26,NaN,16,81,50445,NaN,50445,50445,950790,Taylor Swift,5YqltLsjdqFtvqE7Nrysvs


In [19]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [20]:
print(len(combined_df))

2200


In [21]:
combined_df.to_csv('combined_data/combined.csv', index=False)